[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liuguangw/webui_colab/blob/master/webui.ipynb)

In [ ]:
# @title 1.安装Linux基本环境
!apt -y update
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!apt -y install aria2
%pip install xformers==0.0.16
%pip install triton==2.0.0


In [ ]:
# @title 2.clone webui项目,并挂载云端硬盘

import os
import shutil
from google.colab import drive

%cd /content
repository_path = "/content/stable-diffusion-webui"
# 如果存在则删除目录
!rm -rf {repository_path}
# @markdown 项目地址和git commit hash
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui" # @param {type: "string"}
webui_hash = "22bcc7be428c94e9408f589966c2040187245d81" # @param {type: "string"}
!git clone {repository_url} {repository_path}
%cd {repository_path}
!git checkout {webui_hash}

# @markdown 云端文件夹目录(需要授予权限)
data_dir_name = "sd_webui_colab"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"

drive_path = "/content/drive"

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

models_path = f"{data_dir}/models"
outputs_path = f"{data_dir}/outputs"
settings_path = f"{data_dir}/settings"


if not os.path.exists(models_path):
    shutil.copytree(f"{repository_path}/models", models_path)

if not os.path.exists(outputs_path):
    os.makedirs(outputs_path, exist_ok=True)

if not os.path.exists(settings_path):
    os.makedirs(settings_path, exist_ok=True)

!rm -rf {repository_path}/models
!rm -rf {repository_path}/outputs
!ln -s {models_path} {repository_path}/
!ln -s {outputs_path} {repository_path}/

!rm -rf {repository_path}/venv


In [ ]:
# @title 3.安装扩展(可选操作)
# @markdown 可选扩展
%cd {repository_path}/extensions

ext_tagcomplete = True  # @param {type:"boolean"}
ext_chinese = True  # @param {type:"boolean"}
ext_images_browser = True  # @param {type:"boolean"}
ext_civitai_helper = True  # @param {type:"boolean"}

# 扩展列表
ext_list = [
    (ext_tagcomplete, "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git"),
    (ext_chinese, "https://github.com/VinsonLaro/stable-diffusion-webui-chinese.git"),
    (ext_images_browser,
     "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git"),
    (ext_civitai_helper,
     "https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git"),
]

for install_ext, ext_url in ext_list:
    if install_ext:
        !git clone {ext_url}


In [ ]:
# @title 4.启动webui
%cd {repository_path}
#! ls -al {data_dir}/models
# @markdown 启动参数
up_args = "--xformers --enable-insecure-extension-access --share" # @param {type:"string"}

# @markdown 主题
darkMode = True # @param {type:"boolean"}
if darkMode:
  up_args = f"{up_args} --theme dark"


# @markdown 是否使用ngrok
ngrok = False # @param {type:"boolean"}
ngrok_token = "" # @param {type:"string"}

if ngrok:
  up_args = f"{up_args} --ngrok {ngrok_token}"

up_args = f"{up_args} --ui-settings-file {settings_path}/config.json"
up_args = f"{up_args} --ui-config-file {settings_path}/ui-config.json"
up_args = f"{up_args} --styles-file {settings_path}/styles.csv"

!python launch.py {up_args}